# An Analysis of the Berlin Marathon Runners
- How have the average times changed over the years?
- How have the fastest times changed over the years?
- How does age and gender affect running times
    - Finisher times over the years
- Where are most runners from?
- Where are the fastest runners from?

This project takes the Berlin Marathon dataset and analyzes it look at how running times have changed over the years, and over different age groups and genders. It also takes a look into where the top runners are from.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import panel as pn
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pn.extension('tabulator')

import hvplot.pandas

In [2]:
#format: year, country, gender, age, time
#(all participants)
berlin_marathon = pd.read_csv('1_running_datasets/berlin_marathon_1974_2023.csv')

#data cleaning
berlin_marathon = berlin_marathon[(berlin_marathon['time'] != '–')&
                                  (berlin_marathon['time'] != 'DSQ')&
                                  (berlin_marathon['time'] != 'no time')]
#convert time to hrs
berlin_marathon['time_hrs'] = pd.to_timedelta(berlin_marathon['time']).dt.total_seconds() / 3600
berlin_marathon.dropna(subset = 'time_hrs', how='any')
berlin_marathon

/var/folders/_p/y56kr6vs1fz7ycsbxhyqwwwc0000gn/T/ipykernel_74287/2668505829.py:3: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  berlin_marathon = pd.read_csv('1_running_datasets/berlin_marathon_1974_2023.csv')


,year,country,gender,age,time,time_hrs
0,2023,USA,female,30,05:43:44,5.728889
1,2023,GBR,female,35,03:03:37,3.060278
2,2023,IRL,female,40,02:52:59,2.883056
3,2023,BEL,male,40,03:12:14,3.203889
4,2023,GBR,female,65,04:21:40,4.361111
...,...,...,...,...,...,...
986648,2021,GER,male,30,07:28:28,7.474444
986649,2021,GER,male,30,07:28:29,7.474722
986650,2021,GER,male,80,07:29:14,7.487222
986651,2021,USA,female,20,07:39:21,7.655833


In [3]:
year_slider = pn.widgets.IntSlider(name='Year slider', start=1974, end=2023, step=1, value=2023)
idf = berlin_marathon.interactive()

##### Average and Finishing times
- How have average times changed over the years?
    - Overall average time has increased. Likely due to the increase in interest in the sport.
- How has the fastest time changed over the years?
    - Finishing times have decreased. obvs. because records are meant to be broken.

In [34]:
overall_times = pd.DataFrame(berlin_marathon.groupby('year')['time_hrs'].mean().rename('Average Times'))#.hvplot(kind = 'scatter', title = 'Average times over the years')
overall_times['Finishing times'] = berlin_marathon.groupby('year')['time_hrs'].min()
overall_times_plot = overall_times.hvplot(title = 'Average and Finishing times')
overall_times_plot

:NdOverlay   [Variable]
   :Curve   [year]   (value)

##### Where are most runners from?
- Where do most runners come from?
    - Germany
- Where are the shortest times from?
    - Kenya, followed by Ethiopia. Even tho Germany had the most participants ahaha
- Are these two places the same?
    - no. no they are not.

In [5]:
runners_count = berlin_marathon.groupby(['country', 'gender']).size().sort_values(ascending = False)
runners_count = runners_count[runners_count > 1000]
runners_count_plot = runners_count.hvplot(kind = 'bar', 
                                               stacked = True, 
                                               rot = 45,
                                               title = 'Where are most runners from?')
runners_count_plot

:Bars   [country,gender]   (0)

In [22]:
gender_selector = pn.widgets.Select(options=['All', 'male', 'female'], value='Men', name='Select Gender')
gender_selector

Select(options=['All', 'male', 'female'], value='All')

In [32]:
gender_selector = pn.widgets.Select(options=['male', 'female'], value='Men', name='Select Gender')
top_50_pipeline = (
    idf[
        (idf['gender'] == gender_selector)
    ]
    .dropna(subset=['country', 'gender']).sort_values(by = 'time_hrs')[:50]
    .groupby(['country', 'gender']).size()
    .sort_values(ascending = False)
    .to_frame()
    .reset_index()
    .reset_index(drop=True)
)
top_50_plot = top_50_pipeline.hvplot(kind = 'bar', 
                       x = 'country',
                       ylabel = 'count',
                       stacked = True, 
                       rot = 45,
                       title = 'Where are the top 50 runners from?')

In [75]:
plot_selector = pn.widgets.Select(options=['Top 50', 'Runner Population'], name='Select Plot')
@pn.depends(plot_selector.param.value)
def reactive_plot(selected_plot):
    if selected_plot == 'Top 50':
        return top_50_plot
    elif selected_plot == 'Runner Population':
        return runners_count_plot


In [76]:
template = pn.template.FastListTemplate(
    title = 'Berlin Marathon runners',
    sidebar = [pn.pane.Markdown("# Runners!"),
               year_slider
              ],
    main = [pn.Row(pn.Column(overall_times_plot)),#, top_50_plot)),
           pn.Row(pn.Column(plot_selector, reactive_plot))]
)
template.servable();

In [69]:
reactive_plot

<function __main__.reactive_plot(selected_plot)>